# README.

This notebook is the entrypoint for Azure ML enabled training.
In its essence, it connects to Azure ML, makes sure that everything is ready over there, and starts the training.
To that end, this notebook gathers all necessary sourcecodes in a temp-folder, which will be pushed to Azure ML for training.

# Imports.

In [1]:
%reload_ext autoreload
%autoreload 2

import os
import shutil
from pathlib import Path

import glob2 as glob
from azureml.core import Dataset, Experiment, Environment, Run, ScriptRunConfig, Workspace
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import MpiConfiguration

from src.constants import REPO_DIR
from src.train_util import copy_dir
from src.config import CONFIG, DATASET_MODE_MOUNT, DATASET_MODE_DOWNLOAD

# Setting screws

In [2]:
dataset_name = "anon-depthmap-95k"  # "anon-depthmap-mini"
experiment_name = "training-junkyard-multigpu"
tags = {'run': 'mirrored strategy'}

# Create temp folder and copy code.

Here you have to be very precise, which code to copy.
And most importantly, which code NOT to copy.

In [3]:
code_dir = Path("src")
temp_path = Path("temp_train")
copy_dir(src=code_dir, tgt=temp_path, glob_pattern='*.py')
copy_dir(src=REPO_DIR / "src/common", tgt=temp_path / "temp_common", glob_pattern='*/*.py', should_touch_init=True)

In [ ]:
from temp_train.temp_common.model_utils.environment import cgm_environment

# Connect to azure workspace.

Make sure that you have a config.json file with the keys subscription_id, resource_group, and cgm-ml-dev. Either here (not so nice) or in a parent folder (okay but not perfect), or in the root folder of this repo (way to go).

In [4]:
workspace = Workspace.from_config()
workspace

Workspace.create(name='cgm-ml-prod-we-azml', subscription_id='9b5bbfae-d5d1-4aae-a2ca-75159c0c887d', resource_group='cgm-ml-prod-we-rg')

# Get the experiment.

- You should always arrange all your runs in an experiment.
- Create at least one experiment per sprint.
- Make sure that the name of the experiment reflects the sprint number.
- On top of that you could also add other tokens to the name. For example network architecture, dataset name, and/or targets.

In [5]:
experiment = Experiment(workspace=workspace, name=experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
training-junkyard-multigpu,cgm-ml-prod-we-azml,Link to Azure Machine Learning studio,Link to Documentation


# Find/create a compute target.

Connects to a compute cluster on Azure ML.
If the compute cluster does not exist, it will be created.

Note: Usually computer clusters autoscale. This means that new nodes are created when necessary. And unused VMs will be shut down.

In [6]:
cluster_name = "multigpu-cluster"

# Compute cluster exists. Just connect to it.
try:
    compute_target = ComputeTarget(workspace=workspace, name=cluster_name)
    print("Found existing compute target.")

# Compute cluster does not exist. Create one.    
except ComputeTargetException:
    print("Creating a new compute target...")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='Standard_NC6', 
        max_nodes=4
    )
    compute_target = ComputeTarget.create(workspace, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
compute_target

Found existing compute target.


AmlCompute(workspace=Workspace.create(name='cgm-ml-prod-we-azml', subscription_id='9b5bbfae-d5d1-4aae-a2ca-75159c0c887d', resource_group='cgm-ml-prod-we-rg'), name=multigpu-cluster, id=/subscriptions/9b5bbfae-d5d1-4aae-a2ca-75159c0c887d/resourceGroups/cgm-ml-prod-we-rg/providers/Microsoft.MachineLearningServices/workspaces/cgm-ml-prod-we-azml/computes/multigpu-cluster, type=AmlCompute, provisioning_state=Succeeded, location=westeurope, tags={})

# Get the dataset for training.

Here you specify which dataset to use.

Note: Double check on Azure ML that you are using the right one.

In [7]:
dataset = workspace.datasets[dataset_name]
dataset

{
  "source": [
    "('omdena_datasets', '95k_depthmap_trainingdata/**')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "id": "f2976d42-bc05-4119-8ffa-e369fb9a870c",
    "name": "anon-depthmap-95k",
    "version": 2,
    "description": "A depthmap based dataset containing 95k artifacts, to be used for training.(transfered again after data was missing)",
    "workspace": "Workspace.create(name='cgm-ml-prod-we-azml', subscription_id='9b5bbfae-d5d1-4aae-a2ca-75159c0c887d', resource_group='cgm-ml-prod-we-rg')"
  }
}

# Push the training source code to Azure.

Creates an estimator (a template for a compute cluster node) and pushes it to the compute cluster.

In [8]:
script_params = {f"--{k}": v for k, v in CONFIG.items()}
script_params

{'--DATASET_MODE': 'dataset_mode_download',
 '--DATASET_NAME': 'anon-depthmap-95k',
 '--DATASET_NAME_LOCAL': 'anon-depthmap-mini',
 '--SPLIT_SEED': 0,
 '--IMAGE_TARGET_HEIGHT': 240,
 '--IMAGE_TARGET_WIDTH': 180,
 '--EPOCHS': 1000,
 '--BATCH_SIZE': 256,
 '--SHUFFLE_BUFFER_SIZE': 2560,
 '--NORMALIZATION_VALUE': 7.5,
 '--LEARNING_RATE': 0.0007,
 '--USE_ONE_CYCLE': True,
 '--USE_DROPOUT': False,
 '--USE_WANDB': False,
 '--TARGET_INDEXES': [0],
 '--CODES': ['100', '101', '102', '200', '201', '202']}

In [9]:
cgm_env = cgm_environment(workspace=workspace,curated_env_name='cgm-v30', env_exist=True)

In [10]:
if CONFIG.DATASET_MODE == DATASET_MODE_MOUNT:
    dataset_argument = dataset.as_named_input('cgm_dataset').as_mount()
elif CONFIG.DATASET_MODE == DATASET_MODE_DOWNLOAD:
    dataset_argument = dataset.as_named_input('cgm_dataset').as_download()
else:
    raise Exception("Please specify DATASET_MODE")

In [11]:
# Create the ScriptRunConfig
script_run_config = ScriptRunConfig(source_directory=temp_path,
                                    compute_target=compute_target,
                                    script='train.py',
                                    arguments=[dataset_argument] + [str(item) for sublist in script_params.items() for item in sublist],
                                    environment=cgm_env,
)

# Set compute target.
script_run_config.run_config.target = compute_target

# Run the experiment.
run = experiment.submit(config=script_run_config, tags=tags)

# Show run.
run

Experiment,Id,Type,Status,Details Page,Docs Page
training-junkyard-multigpu,training-junkyard-multigpu_1624605720_378b3a33,azureml.scriptrun,Preparing,Link to Azure Machine Learning studio,Link to Documentation


# Delete temp folder.

After all code has been pushed to Azure ML, the temp folder will be removed.

In [12]:
shutil.rmtree(temp_path)